In [1]:
import pandas as pd

df = pd.read_csv("hf://datasets/MTHR/OCEAN/OCEAN-synthetic.csv")

df.head(10)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


,Text,Openness,Conscientiousness,Extraversion,Agreeableness,Neuroticism
0,I love exploring new cultures through cuisine ...,4.7,3.1,3.5,3.9,2.1
1,My workspace is always organized; I can't focu...,2.9,4.8,2.1,3.2,2.4
2,Large social gatherings make me feel energized...,3.1,2.9,4.6,3.5,1.7
3,I often worry about things not going as planned.,3.0,3.9,2.0,3.4,4.5
4,Having a daily routine is comforting and helps...,2.3,4.6,1.7,3.7,2.2
5,I find fulfillment in helping others with thei...,3.4,3.3,2.8,4.6,2.0
6,Abstract art and avant-garde films usually don...,1.7,3.4,2.7,3.1,2.5
7,I'm the life of the party and always make new ...,3.3,2.2,4.9,3.3,1.8
8,Unfamiliar situations make me anxious and unco...,2.8,3.1,1.4,2.9,4.7
9,Empathy and kindness are my guiding principles...,3.6,3.0,2.4,4.9,2.1


In [21]:
ocean_cols = ["Openness", "Conscientiousness", "Extraversion", "Agreeableness", "Neuroticism"]
binarised_df = df.copy()
for col in ocean_cols:
    median_val = df[col].median()   # we found that the binarisation was more balanced using the median
    binarised_df[col] = (binarised_df[col] >= median_val).astype(int)
binarised_df.head(10)

,Text,Openness,Conscientiousness,Extraversion,Agreeableness,Neuroticism
0,I love exploring new cultures through cuisine ...,1,0,0,1,0
1,My workspace is always organized; I can't focu...,0,1,0,0,0
2,Large social gatherings make me feel energized...,0,0,1,0,0
3,I often worry about things not going as planned.,0,1,0,0,1
4,Having a daily routine is comforting and helps...,0,1,0,0,0
5,I find fulfillment in helping others with thei...,0,0,0,1,0
6,Abstract art and avant-garde films usually don...,0,0,0,0,0
7,I'm the life of the party and always make new ...,0,0,1,0,0
8,Unfamiliar situations make me anxious and unco...,0,0,0,0,1
9,Empathy and kindness are my guiding principles...,0,0,0,1,0


In [4]:
!python -m spacy download en_core_web_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 64.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [22]:
import pandas as pd
import spacy

nlp = spacy.load("en_core_web_md")

def preprocess_text(text):
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
    return " ".join(tokens)

binarised_df["Pre-processed_text"] = binarised_df["Text"].apply(preprocess_text)

binarised_df.head(20)

,Text,Openness,Conscientiousness,Extraversion,Agreeableness,Neuroticism,Pre-processed_text
0,I love exploring new cultures through cuisine ...,1,0,0,1,0,love explore new culture cuisine travel
1,My workspace is always organized; I can't focu...,0,1,0,0,0,workspace organize focus messy environment
2,Large social gatherings make me feel energized...,0,0,1,0,0,large social gathering feel energized excited
3,I often worry about things not going as planned.,0,1,0,0,1,worry thing go plan
4,Having a daily routine is comforting and helps...,0,1,0,0,0,have daily routine comfort help productive
5,I find fulfillment in helping others with thei...,0,0,0,1,0,find fulfillment help problem
6,Abstract art and avant-garde films usually don...,0,0,0,0,0,abstract art avant garde film usually interest
7,I'm the life of the party and always make new ...,0,0,1,0,0,life party new friend social event
8,Unfamiliar situations make me anxious and unco...,0,0,0,0,1,unfamiliar situation anxious uncomfortable
9,Empathy and kindness are my guiding principles...,0,0,0,1,0,empathy kindness guide principle life


In [23]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

vectoriser = TfidfVectorizer()

tfidf_matrix = vectoriser.fit_transform(binarised_df["Pre-processed_text"]) # What is the best way of packaging a pipeline including models from both sklearn and pytorch?

# Convert each row in the TF-IDF matrix to a dense numpy array
tfidf_vectors = [vec.toarray().flatten() for vec in tfidf_matrix]

# Add to DataFrame as a new column
binarised_df["TF-IDF_embeddings"] = tfidf_vectors


In [7]:
print(tfidf_matrix[:100,:100])

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 21 stored elements and shape (100, 100)>
  Coords	Values
  (6, 22)	0.3864555357281473
  (8, 90)	0.4485037125119152
  (14, 40)	0.5218155768742534
  (24, 34)	0.3745553697680986
  (28, 96)	0.5071554866030571
  (33, 90)	0.5710196406963964
  (36, 22)	0.4427903125938777
  (44, 90)	0.5919737985975885
  (46, 18)	0.4430233414544496
  (61, 70)	0.4425337678900139
  (64, 50)	0.42641698683370083
  (65, 48)	0.557071544791232
  (67, 90)	0.45203671608826923
  (68, 59)	0.4686171332658222
  (69, 46)	0.41021279203533995
  (73, 90)	0.4124564898862983
  (83, 89)	0.38022065419481754
  (85, 48)	0.5493670131053857
  (89, 80)	0.3991009693225838
  (93, 89)	0.3563124282435828
  (96, 34)	0.4554589871111926


In [24]:
import numpy as np
import torch
from torch.utils.data import Dataset

class TorchDataset(Dataset):
    def __init__(self, embeddings, targets):
        """
        embeddings: List of np.ndarrays or a np.ndarray of shape (N, embedding_dim)
        targets: List or np.ndarray of shape (N, 5)
        """
        # Efficient conversion
        self.format_inputs = lambda input: torch.tensor(np.stack(input.to_numpy()).astype("float32"), dtype=torch.float32)
        self.embeddings = self.format_inputs(embeddings)
        self.targets = self.format_inputs(targets)

    def __len__(self):
        return len(self.embeddings)

    def __getitem__(self, idx):
        return self.embeddings[idx], self.targets[idx]


In [25]:
import numpy as np

make_ocean_array = lambda row: np.array(row[ocean_cols])

binarised_df["OCEAN_targets"] = binarised_df.apply(func=make_ocean_array, axis=1)
binarised_df.head(10)

,Text,Openness,Conscientiousness,Extraversion,Agreeableness,Neuroticism,Pre-processed_text,TF-IDF_embeddings,OCEAN_targets
0,I love exploring new cultures through cuisine ...,1,0,0,1,0,love explore new culture cuisine travel,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1, 0, 0, 1, 0]"
1,My workspace is always organized; I can't focu...,0,1,0,0,0,workspace organize focus messy environment,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 1, 0, 0, 0]"
2,Large social gatherings make me feel energized...,0,0,1,0,0,large social gathering feel energized excited,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 1, 0, 0]"
3,I often worry about things not going as planned.,0,1,0,0,1,worry thing go plan,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 1, 0, 0, 1]"
4,Having a daily routine is comforting and helps...,0,1,0,0,0,have daily routine comfort help productive,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 1, 0, 0, 0]"
5,I find fulfillment in helping others with thei...,0,0,0,1,0,find fulfillment help problem,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 0, 1, 0]"
6,Abstract art and avant-garde films usually don...,0,0,0,0,0,abstract art avant garde film usually interest,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 0, 0, 0]"
7,I'm the life of the party and always make new ...,0,0,1,0,0,life party new friend social event,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 1, 0, 0]"
8,Unfamiliar situations make me anxious and unco...,0,0,0,0,1,unfamiliar situation anxious uncomfortable,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 0, 0, 1]"
9,Empathy and kindness are my guiding principles...,0,0,0,1,0,empathy kindness guide principle life,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 0, 1, 0]"


In [10]:
type(np.stack(binarised_df["OCEAN_targets"].to_numpy()).astype("float32")[4,4])

numpy.float32

In [26]:
dataset = TorchDataset(binarised_df["TF-IDF_embeddings"],
                       binarised_df["OCEAN_targets"])

In [27]:
from torch.utils.data import DataLoader, random_split

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=True)

In [28]:
import torch.nn as nn
from torch import optim

In [29]:
len(binarised_df.loc[0, "TF-IDF_embeddings"])

1774

In [30]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(1774, 20)
        self.fc2 = nn.Linear(20, 8)
        self.fc3 = nn.Linear(8, 5) # Five OCEAN scores out
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x) # No activation function here: We want to use the sigmoid built into the loss func during training.
        return x

model = Model()
print(model)

Model(
  (fc1): Linear(in_features=1774, out_features=20, bias=True)
  (fc2): Linear(in_features=20, out_features=8, bias=True)
  (fc3): Linear(in_features=8, out_features=5, bias=True)
  (relu): ReLU()
)


TODO:
 - Essayer avec "enlever les stop words" | "lemmatiser" | embedding
 - Essayer sans nettoyage du tout
 -

In [31]:
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCEWithLogitsLoss()

In [ ]:
import torch

def train(model, train_loader, val_loader, criterion, optimizer, epochs=100):
    """
    Function to train a PyTorch model with training and validation datasets.

    Parameters:
    model: The neural network model to train.
    train_loader: DataLoader for the training dataset.
    val_loader: DataLoader for the validation dataset.
    criterion: Loss function (e.g., Binary Cross Entropy for classification).
    optimizer: Optimization algorithm (e.g., Adam, SGD).
    epochs: Number of training epochs (default=100).

    Returns:
    history: Dictionary containing loss and accuracy for both training and validation.
    """


    history = {'loss': [], 'val_loss': [], 'full_accuracy': [], 'val_full_accuracy': [], 'partial_accuracy': [], 'val_partial_accuracy': []}

    for epoch in range(epochs):
        model.train()
        total_loss, fully_correct, partially_correct = 0, 0, 0

        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs).squeeze()
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            sigmoided_outputs = torch.sigmoid(outputs) >= 0.5

            total_loss += loss.item()
            fully_correct += torch.equal(sigmoided_outputs, labels)
            partially_correct += (sigmoided_outputs == labels).sum().item()

        train_loss = total_loss / len(train_loader)
        train_full_acc = fully_correct / len(train_loader.dataset)
        train_part_acc = (partially_correct / 5) / len(train_loader.dataset) # partially_correct keeps count of count every instance of a correctly-guessed element in a vector,
                                                                             # so potentially up to 5 per vector. Hence the division by 5.
        model.eval()
        val_loss, val_fully_correct, val_partially_correct = 0, 0, 0
        with torch.no_grad():
            for inputs, labels in val_loader:
               outputs = model(inputs).squeeze()
               loss = criterion(outputs, labels)
               sigmoided_outputs = torch.sigmoid(outputs) >= 0.5

               val_loss += loss.item()
               val_fully_correct += torch.equal(sigmoided_outputs, labels)
               val_partially_correct += (sigmoided_outputs == labels).sum().item()

        val_loss = total_loss / len(train_loader)
        val_full_acc = fully_correct / len(train_loader.dataset)
        val_part_acc = (partially_correct / 5) / len(train_loader.dataset) # partially_correct keeps count of every instance of a correctly-guessed element in a vector,
                                                                           # so potentially up to 5 per vector. Hence the division by 5.
        history['loss'].append(train_loss)
        history['val_loss'].append(val_loss)
        history['full_accuracy'].append(train_full_acc)
        history['val_full_accuracy'].append(val_full_acc)
        history['partial_accuracy'].append(train_part_acc)
        history['val_partial_accuracy'].append(val_part_acc)

        print(f"Epoch [{epoch+1}/{epochs}], "
              f"Loss: {train_loss:.4f}, Full acc.: {train_full_acc:.4f}, Partial acc.: {train_part_acc:.4}, "
              f"Val. Loss: {val_loss:.4f}, Val. full acc.: {val_full_acc:.4f}, Val. partial acc.: {val_part_acc:.4f}")
    return history

In [48]:
history = train(model, train_loader, val_loader, criterion, optimizer, epochs=50)

Epoch [1/50],Loss: 0.0137, Full acc.: 0.0399, Partial acc.: 0.9929,Val. Loss: 0.0137, Val. full acc.: 0.0399, Val. partial acc.: 0.9929
Epoch [2/50],Loss: 0.0142, Full acc.: 0.0420, Partial acc.: 0.9933,Val. Loss: 0.0142, Val. full acc.: 0.0420, Val. partial acc.: 0.9933
Epoch [3/50],Loss: 0.0138, Full acc.: 0.0409, Partial acc.: 0.9938,Val. Loss: 0.0138, Val. full acc.: 0.0409, Val. partial acc.: 0.9938
Epoch [4/50],Loss: 0.0135, Full acc.: 0.0420, Partial acc.: 0.9944,Val. Loss: 0.0135, Val. full acc.: 0.0420, Val. partial acc.: 0.9944
Epoch [5/50],Loss: 0.0133, Full acc.: 0.0431, Partial acc.: 0.994,Val. Loss: 0.0133, Val. full acc.: 0.0431, Val. partial acc.: 0.9940
Epoch [6/50],Loss: 0.0131, Full acc.: 0.0409, Partial acc.: 0.9942,Val. Loss: 0.0131, Val. full acc.: 0.0409, Val. partial acc.: 0.9942
Epoch [7/50],Loss: 0.0137, Full acc.: 0.0420, Partial acc.: 0.9935,Val. Loss: 0.0137, Val. full acc.: 0.0420, Val. partial acc.: 0.9935
Epoch [8/50],Loss: 0.0134, Full acc.: 0.0420, Par

In [ ]:
# Our criterion nn.BCEWithLogitsLoss() has a sigmoid function built in, so we left out the sigmoid in the model.
# This means we need to manually run the output through a sigmoid function when making predictions on a "finished" model.

user_input = "this is a placeholder for the data input by the user when the model is deployed"

with torch.no_grad():
    logits = model(user_input)
    probs = torch.sigmoid(logits)
    predictions = (probs > 0.5).int()